# SANS2D: reducing measurement and backgroud to Q

In [ ]:
import scipp as sc
from ess import loki, sans
import scippneutron as scn

## Loading data files

In [ ]:
path = 'SANS2D_data'

sample_run_number = 63114
direct_run_number = 63091
background_run_number = 63159

In [ ]:
ds = sc.Dataset()

#Using only one-forth of the full spectra 245760 (reserved for first detector)
spectrum_size =  245760//4

# Make common tof bins so that dataset items are aligned
tof_bins = sc.linspace(dim='tof', start=0, stop=100000, num=2, unit='us')
wavelength_bins = sc.linspace(dim='wavelength', start=2.0, stop=16.0, num=141, unit='angstrom')
q_bins = sc.linspace(dim='Q', start=0.01, stop=0.6, num=141, unit='1/angstrom')

# Sample measurement
ds['sample'] = loki.io.load_sans2d(filename=f'{path}/SANS2D000{sample_run_number}.nxs',
                                   spectrum_size=spectrum_size, tof_bins=tof_bins)
# Direct measurement is with the empty sample holder/cuvette
ds['direct'] = loki.io.load_sans2d(filename=f'{path}/SANS2D000{direct_run_number}.nxs',
                                   spectrum_size=spectrum_size, tof_bins=tof_bins)
# Background is a measurement with the solvent which the sample is placed in
ds['background'] = loki.io.load_sans2d(filename=f'{path}/SANS2D000{background_run_number}.nxs',
                                       spectrum_size=spectrum_size, tof_bins=tof_bins)

In [ ]:
ds

## Apply offsets to pixel positions

In [ ]:
# Custom SANS2D position offsets
sample_pos_z_offset = 0.053 * sc.units.m
bench_pos_y_offset = 0.001 * sc.units.m
# There is some uncertainity here
monitor4_pos_z_offset = -6.719 * sc.units.m

# Geometry transformation
x_offset = -0.09288 * sc.units.m
y_offset = 0.08195 * sc.units.m
# z_offset = 0.0 * sc.units.m

In [ ]:
ds.coords["pixel_width"] = 0.0035 * sc.units.m
ds.coords["pixel_height"] = 0.002033984375 * sc.units.m

# Change sample position
ds.coords["sample_position"].fields.z += sample_pos_z_offset
# Apply bench offset to pixel positions
ds.coords["position"].fields.y += bench_pos_y_offset

for key in ds:
    ds[key].attrs["monitor4"].value.coords["position"].fields.z += monitor4_pos_z_offset

# Now shift pixels positions to get the correct beam center
ds.coords['position'].fields.x += x_offset
ds.coords['position'].fields.y += y_offset

## Mask bad pixels

In [ ]:
mask_edges = (
    (sc.abs(ds.coords['position'].fields.x - x_offset) > sc.scalar(0.48, unit='m')) |
    (sc.abs(ds.coords['position'].fields.y - y_offset) > sc.scalar(0.45, unit='m')))

mask_center = sc.sqrt(
    ds.coords['position'].fields.x**2 +
    ds.coords['position'].fields.y**2) < sc.scalar(0.04, unit='m')
# cylindrical_radius = sc.sqrt(
#     ds.coords['position'].fields.x**2 +
#     ds.coords['position'].fields.y**2)
# mask_center = (cylindrical_radius < sc.scalar(0.04, unit='m')) | (cylindrical_radius > sc.scalar(0.4, unit='m'))

for key in ds:
    ds[key].masks['edges'] = mask_edges
    ds[key].masks['center'] = mask_center

In [ ]:
scn.instrument_view(ds['sample'], pixel_size=0.0075)

## Load detector efficiency function (measured from direct beam)

In [ ]:
# Load efficency correction for main detector
detector_efficiency_file = 'DIRECT_SANS2D_REAR_34327_4m_8mm_16Feb16.dat'
detector_efficiency = loki.io.load_rkh_wav(filename=f'{path}/{detector_efficiency_file}')
# Crop (or rebin?) to relevant range
detector_efficiency = detector_efficiency['wavelength', wavelength_bins.min():wavelength_bins.max()].copy(deep=True)
#
# for attr in list(detector_efficiency.attrs.keys()):
#     del detector_efficiency.attrs[attr]

## Use Q1D workflow

In [ ]:
sample_q = sans.q1d(data=ds['sample'],
        data_incident_monitor=ds['sample'].attrs["monitor2"].value,
        data_transmission_monitor=ds['sample'].attrs["monitor4"].value,
        direct_incident_monitor=ds['direct'].attrs["monitor2"].value,
        direct_transmission_monitor=ds['direct'].attrs["monitor4"].value,
        detector_efficiency=detector_efficiency,
        wavelength_bins=wavelength_bins,
        q_bins=q_bins,
        monitor_background_threshold=sc.scalar(30.0, unit='counts'))
sample_q

In [ ]:
sc.plot(sample_q)

In [ ]:
background_q = sans.q1d(data=ds['background'],
        data_incident_monitor=ds['background'].attrs["monitor2"].value,
        data_transmission_monitor=ds['background'].attrs["monitor4"].value,
        direct_incident_monitor=ds['direct'].attrs["monitor2"].value,
        direct_transmission_monitor=ds['direct'].attrs["monitor4"].value,
        detector_efficiency=detector_efficiency,
        wavelength_bins=wavelength_bins,
        q_bins=q_bins,
        monitor_background_threshold=sc.scalar(30.0, unit='counts'))

In [ ]:
background_q.plot()